# Introduktion til sentiment analysis

## Hvad er sentiment analysis?

> "[...] the field of study that analyzes people’s opinions, sentiments, appraisals, attitudes, and emotions toward entities and their attributes expressed in written text" - Liu 2015

**Formål:** Identificere positive og/eller negative holdninger/følelser i tekst samt hvad, disse holdninger/følelser er rettet imod.

**Eksempler på brug:**

- Undersøge sociale bevægelser på internettet
- Markedsprædiktion
- Policy-evaluering

**Relateret:** opinion mining, opinion analysis, opinion extraction, sentiment mining, subjectivity analysis, affect analysis, emotion analysis, review mining

## Sentiment analysis - Tre overordnede teknikker

**A: Ordbogs- og regelbaseret sentiment analysis ("lexicon and rule-based")**

- Model der grundlæggende bygger på et opslagsværk over, hvordan et ord er følelsesmæssigt ladet (fx "hader" -> negativt, "elsker" -> positivt)
- Kan evt. udbygges med regler sådan, at der tages højde for ord i kontekst (fx "hader ikke" -> neutralt)
- Fordele: inspicérbar, alsidigt
- Ulemper: tager i meget begrænset omfang højde for kontekst

**B: Præ-trænede sentiment classifiers**

- Model der er trænet på eksisterende tekststykker, som er tagget efter holdningen/følelsen udtrykt i tekststykket (fx positivt, negativt, neutralt)
- Gør brug af machine learning i en eller anden omfang (random forests, naive bayes, neural networks)
- Fordele: tager i højere grad højde for kontekst, (ofte) stort træningsdatasæt
- Ulemper: vanskelig at inspicere

**C: Selv-trænede sentiment classifiers**

- Model der trænes på taggede tekststykker i det materiale, som skal undersøges
- Gør brug af analytikerens egen kendskab til området og materialet til at udlede sentiment
- Fordele: høj kontekstnøjagtighed, til dels inspicerbar
- Ulemper: (sandsynligvis) lille træningsdatasæt, kræver manuel tagging

**Generelle svagheder**

- Kontekst: Modeller er for det meste trænet på data, der deler en eller anden kontekst (tid, genre, sprog)
- Ironi/sarkasme: Vanskeligt at træne modeller til at genkende dette ud fra tekst alene

## Sentiment analysis i Python 1: Data

Vi starter med at importere de datasæt, som vi skal arbejde med i dag.

In [ ]:
import pandas as pd

In [ ]:
tweetdata_url = "https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/poltweets_sample.csv"
tweets_df = pd.read_csv(tweetdata_url)

redditdata_url = "https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv"
reddit_df = pd.read_csv(redditdata_url)

**Datasæt 1: Danske politikere på Twitter**

Det første datasæt er en samling af tweets fra danske folketingspolitikere. Der er tweets fra perioden 1/1/2015 - 27/1/2021. Data er et sample bestående af 500 tilfædige tweets per danske parti i den periode. Personlige oplysninger i form af brugernavn, placering og brugerbeskrivelser er fjernet fra data.

Dette datasæt bliver brugt i de fleste eksempler i løbet af dagens session.

In [ ]:
tweets_df.head()

**Datasæt 2: Kommentarer fra Reddit (r/denmark)**

Det andet datasæt er et udtræk af Reddit-opslag med kommentarer fra subreddit [r/denmark](https://www.reddit.com/r/Denmark/). Udtrækket består af opslag fra 1/3/2021-8/3/2021 med mere end 15 kommentarer på udtrækstidspunktet (8/3/2021).

Dette datasæt bliver brugt i dagens øvelser.

In [ ]:
reddit_df.head()

## Sentiment analysis i Python 1: DaCy

En implementation af sentiment analysis i Python er gennem DaCy: https://centre-for-humanities-computing.github.io/DaCy/

DaCy er en komplet dansk sprogmodel-løsning baseret på spaCy. DaCy indeholder en såkaldt "wrapper" til `senda`, som er en sentiment analyzer baseret på en *transformer-model (neural netværk)*.

`senda`-modellen er udviklet af EkstraBladet og er trænet på twitter-data: https://github.com/ebanalyse/senda

### Download og indlæsning

DaCy fungerer mere eller mindre præcist ligesom spaCy. For at bruge den, gør vi følgende:

1. Indlæser DaCy
2. Downloader sprogmodel
3. Indlæser sprogmodel
4. Tilføjer senda til modellens pipeline

In [ ]:
import dacy
from dacy.sentiment import add_senda

In [ ]:
nlp = dacy.load("da_dacy_medium_tft-0.0.0")
add_senda(nlp, force_extension = True)

### Brug af sentiment analyzer

Sentiment analyzeren bruges på et "doc" objekt (et spaCy dokument - mere om dette senere). Vi gør følgende:

- Danner spaCy dokument (med sprogmodel fra DaCy)
- Undersøger sentiment (under attribut `._.polarity` og `._.polarity_prop`)

In [ ]:
doc = nlp("Den her kaffe er den værste gang tynde pis")

print(doc.text,
      "\n",
      doc._.polarity, 
      "\n",
      doc._.polarity_prop)

### Sentiment analyzer på dataframe

Ved brug af en kort wrapper-funktion og `.apply` kan sentiment analysis udføres på tekststykker i en dataframe. Vi gør følgende:

1. Danner wrapper-funktion til sentiment-analyse
2. Danner subset af twitter-data (Folketingsmedlemmers tweets)
3. Bruger funktion med apply

In [ ]:
tweets_df.head()

In [ ]:
def simple_sentiment(text):
    doc = nlp(text)
    polarity = doc._.polarity
    
    return(polarity)

In [ ]:
tweets_alternativet = tweets_df.loc[tweets_df['party'] == 'Alternativet', :]
tweets_alternativet['polarity'] = tweets_alternativet['full_text'].apply(simple_sentiment)

tweets_alternativet.head()

In [ ]:
tweets_alternativet['polarity'].value_counts()

## Sentiment analysis  i Python 2: TextBlob

En implementation af sentiment analysis i Python er med TextBlob: https://textblob.readthedocs.io/en/

TextBlob er en sentiment analyzer model *baseret på naive bayes (machine learning)*,

**Bemærk:** TextBlob egner sig primært til engelsk.

### Indlæsning og brug

TextBlob fungerer på mange måder ligesom spaCy. Man omdanner tekststykket til et "blob" objekt, der indeholder forskellige attributter - herunder sentiment. Vi gør følgende

1. Indlæser TextBlob
2. Danne "blob" objekt ud fra tekststykke
3. Undersøge sentiment (under attribut `sentiment`)

In [ ]:
from textblob import TextBlob

blob = TextBlob("This coffee is just absolutely horrible. I hate it.")
print(blob.sentiment)

Sentiment-scoren ses af "polarity = -0.9". Scoren går fra -1 til 1.

### TextBlob i funktion

Ovenstående kan nemt konverteres til funktion, som fx kunne bruges i en dataframe med `.apply`. 

Sentiment-outputtet fra TextBlob kan konverteres til en tuple, hvor den første værdi er polarity-scoren.

In [ ]:
def simple_polarity(text):
    blob = TextBlob(text)
    polarity = tuple(blob.sentiment)[0]
    
    return(polarity)

In [ ]:
simple_polarity("What an absolutely amazing weekend!")

## ØVELSE 1: Sentiment Analysis

Foretag sentiment analysis enten på eget data eller på r/Denmark data: [reddit_rdenmark-comments_01032021-08032021_long.csv](https://raw.githubusercontent.com/CALDISS-AAU/course_ddf/master/datasets/reddit_rdenmark-comments_01032021-08032021_long.csv)

(DaCy/senda for dansk, TextBlob for engelsk)

**Bemærk:** Sentiment analysis med især DaCy kan tage lang tid på større datasæt. I kan med fordel arbejde med et subset i denne øvelse, for at teste funktionerne af.

1. Anvend sentiment analysis på enkelte tekststykker
2. Anvend sentiment analysis på et subset (enten med egne betingelser eller med [`pd.sample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html))
3. Foretag opsummering af subsettet, der giver indikation af, hvorvidt tonen i materialet er overvejende positiv eller negativ